In [1]:

%load_ext autoreload 
# %aimport rl_envs.grid_world_env

%autoreload 2
import math

from agents.approx_Q_Learning import ApproxQLearningAgent
from tools.helper import *
import  gymnasium  as gym
from rl_envs.new_gym_grid_world_env import GridWorldEnv
from collections import defaultdict
from torch.utils.tensorboard import SummaryWriter # type: ignore
import itertools
from agents.approx_Q_Learning_cliff import CliffApproxQLearningAgent

%autoreload 2


pygame 2.5.2 (SDL 2.28.3, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
DISCOUNTED_FACTOR = 0.999
FORBIDDEN_REWARD = -10
HITWALL_REWARD = -10
TARGET_REWARD = 10
NORMAL_REWARD = 0

SEED = 666

In [3]:
# env = GridWorldEnv(
#     size=3,
#     fixed_map=True,
#     forbidden_grids=[(1, 1), (1, 2)],
#     target_grids=[(2, 2)],
#     forbidden_reward=FORBIDDEN_REWARD,
#     hit_wall_reward=HITWALL_REWARD,
#     target_reward=TARGET_REWARD,
#     normal_reward=NORMAL_REWARD,
# )
env = GridWorldEnv(fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD, normal_reward=NORMAL_REWARD)


In [22]:
agent = ApproxQLearningAgent(action_space_n=env.action_n)

agent.RUN(env, num_episodes=100000, episode_len=20000)
# agent.RUN(env, num_episodes=10000, episode_len=2000)
# num_episodes=1000, episode_len=2000 能解出 3x3

d:\apps\anaconda\envs\finRL_310\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.action_n to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.action_n` for environment variables or `env.get_wrapper_attr('action_n')` that will search the reminding wrappers.
  logger.warn(


AttributeError: 'CliffWalkingEnv' object has no attribute 'action_n'

In [19]:
for y in range(env.height):
    for x in range(env.width):
        agent.policy_improvement((y, x))

Q = defaultdict(lambda: [0 for _ in range(agent.action_space_n)])
for y in range(env.height):
    for x in range(env.width):
        for action in range(agent.action_space_n):
            state = (y,x)
            Q[state][action] = agent.estimate_q(state, action)
print_by_dict(env, Q)

# ISSUE: 在 X 相同时, 不同的 y, q 值保持一致 (为何)


[ [-0.3115143504536536, 0.15373168652040714, -9.701163625904208, -9.098343567776727, 0.6869060448910653] [-9.476055695663948, -0.37252766887615013, -11.881987367989673, 0.504187350768549, 0.5172676537618375] [-8.283195280190442, 0.25701268280685186, -9.251251770247457, -0.3862925727061258, -0.5515061735954295] [0.2433918703351825, 0.23970791447891115, -9.446275072248474, 0.04921812597993702, -0.24151862779083055] [0.2263374577724978, -9.687782353409737, -9.693326802837177, 0.20360609309668587, 0.09310984988765414] ]
[ [-6.578993214290452, -12.078773398723179, 0.3331657189021575, -8.069392793026942, -6.945706495767432] [0.7465380445216161, -8.211677314307792, -0.938744271482185, -2.434694494312684, -8.175657938313941] [-3.3728513895807612, -0.4766312343022945, 0.5216383194818321, -12.855477457219202, -13.911099744941072] [0.13457090212703504, 0.22836794018654127, 0.21701317851239904, -9.481618533320407, 0.12815750958615868] [0.05018854115270577, -9.772494674214908, 0.20368489661270628, 

In [20]:
print_by_dict(env, agent.policy)
print()
print(agent.parameters)

print(env)
for i in range(env.height):
    print("[", end=" ")
    for j in range(env.width):
        state = (i, j)
        action = agent.get_action(state, optimal=True)
        print(env.action_mappings[action], end=" ")
    print("]")

[ [0.02 0.02 0.02 0.02 0.92] [0.02 0.02 0.02 0.02 0.92] [0.02 0.92 0.02 0.02 0.02] [0.92 0.02 0.02 0.02 0.02] [0.92 0.02 0.02 0.02 0.02] ]
[ [0.02 0.02 0.92 0.02 0.02] [0.92 0.02 0.02 0.02 0.02] [0.02 0.02 0.92 0.02 0.02] [0.02 0.92 0.02 0.02 0.02] [0.02 0.02 0.02 0.92 0.02] ]
[ [0.02 0.02 0.02 0.02 0.92] [0.02 0.02 0.02 0.92 0.02] [0.02 0.02 0.02 0.92 0.02] [0.02 0.02 0.92 0.02 0.02] [0.02 0.02 0.02 0.92 0.02] ]
[ [0.92 0.02 0.02 0.02 0.02] [0.02 0.02 0.92 0.02 0.02] [0.02 0.02 0.02 0.92 0.02] [0.02 0.92 0.02 0.02 0.02] [0.02 0.02 0.02 0.02 0.92] ]
[ [0.02 0.02 0.02 0.02 0.92] [0.02 0.92 0.02 0.02 0.02] [0.02 0.02 0.02 0.02 0.92] [0.02 0.92 0.02 0.02 0.02] [0.02 0.02 0.02 0.02 0.92] ]

[-4.57554407  0.55017997  0.72047626 -0.69717129 -1.6892028  -0.7497669
  0.11583587  0.24043409 -0.70021934  0.0914042   2.15549114  0.89401434
  1.71693936  1.67135583 -0.43977293 -0.13612851 -0.53958727  0.97631375
  1.71196945 -1.071385   -1.411142    0.43318891 -0.52883288 -0.61320027
 -2.47908842 

In [7]:
env = gym.make("CliffWalking-v0")
agent = CliffApproxQLearningAgent(action_space_n=env.action_space.n, discounted_factor=0.99999)

agent.RUN(env, num_episodes=100000, episode_len=20000)

Step 0 @ Episode 1/100000 (-100.0, 0.0)
 len 0 Episode 0/100000 ( TD_error: -100.0, reward: -100.0,0.0)
Step 0 @ Episode 2/100000 (-100.0, -100.0)
Step 0 @ Episode 3/100000 (-1.0, -100.0)
Step 0 @ Episode 4/100000 (-1.0, -123.0)
Step 0 @ Episode 5/100000 (-1.0, -120.0)
Step 0 @ Episode 6/100000 (-1.0, -128.0)
Step 0 @ Episode 7/100000 (-1.0, -111.0)
Step 0 @ Episode 8/100000 (-1.0, -115.0)
Step 0 @ Episode 9/100000 (-1.0, -117.0)
Step 0 @ Episode 10/100000 (-1.0, -241.0)
Step 0 @ Episode 11/100000 (-1.0, -120.0)
Step 0 @ Episode 12/100000 (-1.0, -275.0)
Step 0 @ Episode 13/100000 (-1.0, -146.0)
Step 0 @ Episode 14/100000 (-1.0, -193.0)
Step 0 @ Episode 15/100000 (-1.0, -385.0)
Step 0 @ Episode 16/100000 (-1.0, -114.0)
Step 0 @ Episode 17/100000 (-1.0, -131.0)
Step 0 @ Episode 18/100000 (-1.0, -157.0)
Step 0 @ Episode 19/100000 (-1.0, -339.0)
Step 0 @ Episode 20/100000 (-1.0, -369.0)
Step 0 @ Episode 21/100000 (-1.0, -142.0)
Step 0 @ Episode 22/100000 (-1.0, -285.0)
Step 0 @ Episode 23/

KeyboardInterrupt: 

In [8]:
action_mappings = ["↑","→","↓","←"]
for i in range(48):
    if i % 12 == 0:
        print()
    action = agent.get_action(i)
    print(action_mappings[action], end=" ")


↑ → → → ↓ ← ← ← → ↓ ↑ ↑ 
↑ → ← ↓ ← ← ← ↑ ← ← ↑ → 
↓ ← ↓ ↑ → ↓ ↓ ↓ ← ↓ ← → 
← ↑ ↑ ← ← → ↑ ↑ ← ← → ↓ 